# Generating interesting visualizations from the results of QUAL-PROB

In [2]:
import numpy as np
import pandas as pd

from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.offline as pyo

from os import listdir, path
import json

from utils import Grapher, GraphHelper

In [3]:
parameters = json.load(open("config.json"))

In [5]:
scenarios = ['current_scenario', 'scenario_1', 'scenario_2']

In [6]:
parameters_df = pd.DataFrame(parameters)
parameters_df = parameters_df.set_index("name", drop=True)
pars_list = parameters_df['val'].to_list()

In [7]:
data_folders = ["./data/current_scenario/raw/", "./data/scenario_1/raw/",
                "./data/scenario_2/raw/"]
dfs = list()

for i, folder in enumerate(data_folders):
    df_data = list()
    for par in parameters_df['val']:
        df_data.append(pd.read_csv(folder + par + '.csv',
                                  header = None,
                                  names = list(range(1, 1001)),
                                  usecols = list(range(0, 1000)),
                                  nrows = 435,
                                  dtype = np.float64))
    dfs.append(df_data)

In [8]:
folder_obs = './data/obs'
df_min = pd.read_csv(path.join(folder_obs, 'obs_min.csv'), index_col = 0)
df_max = pd.read_csv(path.join(folder_obs, 'obs_max.csv'), index_col = 0)
df_min.drop('E', axis = 0, inplace = True)
df_max.drop('E', axis = 0, inplace = True)

In [9]:
det_folder = './data/det/'
df_dets = list()
for scenario in scenarios:
    df_data = pd.read_csv(
        det_folder + scenario + ".csv",
        dtype = np.float64)
    df_dets.append(df_data)

In [10]:
g = Grapher(
    parameter="bod", 
    df=dfs[0][pars_list.index("bod")], 
    limit=parameters_df.loc["BOD"]["limit"], 
    df_obs = [df_min[["x", "bod"]], df_max[["x", "bod"]]],
    df_det = df_dets[0][["x", "bod"]])

In [56]:
g.update_bar_point(point=25)

In [57]:
g.update_scatter_plot(members=10)

In [58]:
g.plot_accordance()

## All parameters in scenario 0
Note: exports to html

In [61]:
GraphHelper(config=parameters, dfs=dfs[0], df_obs=df_obs, df_det=df_dets[0]).plot_all()

## Accordance for all scenarios

In [12]:
import itertools

names = []
for p in itertools.product(
    parameters_df.index, ["Current Scenario", "Scenario 1", "Scenario 2"]
):
    names.append(f"{p[0]} {p[1]}")

fig = make_subplots(
    rows=8,
    cols=3,
    subplot_titles=names,
    shared_xaxes=False,
    shared_yaxes=True,
    vertical_spacing=0.08,
    horizontal_spacing=0.06,
)

x_vals = len(dfs[0][0]) / 10  # in km
limits = parameters_df["limit"]

xaxis_layout = dict(
    title=dict(text="Distance [km]", standoff=0.5, font=dict(size=12)),
    dtick=10,
    range=(0, x_vals),
)
yaxis_layout = dict(
    title=dict(text="PNA (%)", standoff=0.5, font=dict(size=12)), range=(0, 101)
)

layout = dict(
    width=1000,
    height=1400,
    showlegend=True,
    legend=dict(title="Legend", yanchor="bottom", y=0.1, xanchor="left", x=0.52),
)


for s, scenario in enumerate(scenarios):
    for i, param in enumerate(parameters_df["val"].to_list()):
        g = Grapher(
            parameter=param,
            df=dfs[s][i],
            limit=limits[i],
            df_det=df_dets[s][["x", param]],
        )
        plot_fig = g.plot_accordance().data
        plot_indx = (i + 1, s + 1)
        for trace in plot_fig:
            if i > 0:
                trace["showlegend"] = False
            if s > 0:
                trace["showlegend"] = False
            fig.add_trace(trace, plot_indx[0], plot_indx[1])

for i in range(26):
    axis_num = str(i + 1) if i > 0 else ""
    layout[f"yaxis{axis_num}"] = yaxis_layout
    layout[f"xaxis{axis_num}"] = xaxis_layout

layout = go.Layout(**layout)
fig.update_layout(layout)
fig.layout.template = "simple_white"
fig.update_layout(
    legend=dict(title="Legend", yanchor="bottom", y=0.01, xanchor="left", x=0.82)
)
fig.update_yaxes(rangemode="tozero")
fig.show()
# pyo.plot(fig, filename = path.join(f"images/", str(scenario) + '.html'))
# fig.write_image(path.join(f"images/", str(scenario) + '.png'))